# Access The Cleaned DataFrames

In [ ]:
# Access the global temporary views
df_pin_cleaned = spark.read.table("global_temp.global_cleaned_pin")
df_geo_cleaned = spark.read.table("global_temp.global_cleaned_geo")
df_user_cleaned = spark.read.table("global_temp.global_cleaned_user")

In [ ]:
# Display Cleaned Pinterest Data
display(df_pin_cleaned)

In [ ]:
# Display Cleaned Geo Data
display(df_geo_cleaned)

In [ ]:
# Display Cleaned User Data
display(df_user_cleaned)

# Query The DataFrames

## **Make sure to optimise and refactor the code in this notebook to avoid uneccessary repetition: DRY CODING**

In [ ]:
# Import the necessary classes and functions from the PySpark SQL module
from pyspark.sql import Window
from pyspark.sql.functions import count, col, row_number, year, max, when, expr, percentile_approx

In [ ]:
# M7: Task 4
# Find the most popular Pinterest category in each country

# Join df_pin_cleaned with df_geo_cleaned on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo_cleaned, on="ind", how="inner")

# Group by 'country' and 'category' and count the number of posts in each category per country
df_category_count = df_combined.groupBy("country", "category") \
                               .agg(count("category").alias("category_count"))

# Define a window specification to rank categories by count within each country
window_spec = Window.partitionBy("country").orderBy(col("category_count").desc())

# Apply the window function to add a rank column
df_ranked = df_category_count.withColumn("rank", row_number().over(window_spec))

# Filter to keep only the most popular category per country (rank == 1)
df_most_popular = df_ranked.filter(col("rank") == 1).drop("rank")

# Display the final result
display(df_most_popular)

In [ ]:
# M7: Task 5
# Find the most popular category in each year (between 2018 - 2022)

# Join df_pin_cleaned with df_geo_cleaned on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo_cleaned, on="ind", how="inner")

# Extract the year from the timestamp to create 'post_year' column
df_pin_with_year = df_combined.withColumn("post_year", year(col("timestamp")))

# Filter the data to include only posts between 2018 and 2022
df_filtered = df_pin_with_year.filter(col("post_year").between(2018, 2022))

# Group by 'post_year' and 'category' and count the number of posts in each category per year
df_category_count = df_filtered.groupBy("post_year", "category") \
                               .agg(count("category").alias("category_count"))

# Define a window specification to rank categories by count within each year
window_spec = Window.partitionBy("post_year").orderBy(col("category_count").desc())

# Apply the window function to add a rank column
df_ranked = df_category_count.withColumn("rank", row_number().over(window_spec))

# Filter to keep only the most popular category per year (rank == 1)
df_most_popular = df_ranked.filter(col("rank") == 1).drop("rank")

# Display the final result
display(df_most_popular)

In [ ]:
# M7: Task 6
# Find the user with the most followers in each country.

# Join df_pin_cleaned, df_geo_cleaned, and df_user_cleaned on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo_cleaned, on="ind", how="inner") \
                            .join(df_user_cleaned, on="ind", how="inner")

# Group by 'country' and 'user_name' and find the maximum follower count
df_max_followers = df_combined.groupBy("country", "user_name") \
                              .agg(max("follower_count"))

# Define a window specification to rank users by follower count within each country
window_spec = Window.partitionBy("country").orderBy(col("max(follower_count)").desc())

# Apply the window function to add a rank column
df_ranked = df_max_followers.withColumn("rank", row_number().over(window_spec))

# Filter to keep only the user with the most followers per country (rank == 1)
df_top_users = df_ranked.filter(col("rank") == 1).select("country", "user_name", "max(follower_count)")

# Display the result for Step 1
display(df_top_users)

# Define a window specification to rank countries by the top user's follower count
window_spec_country = Window.orderBy(col("max(follower_count)").desc())

# Apply the window function to rank countries and filter for the top-ranked country
df_top_country = df_top_users.withColumn("rank", row_number().over(window_spec_country)) \
                             .filter(col("rank") == 1) \
                             .select("country", "max(follower_count)")

# Display the result for Step 2
display(df_top_country)


In [ ]:
# M7: Task 7
# Find the most popular category for different age groups

# Create an 'age_group' column based on the 'age' column
df_with_age_group = df_combined.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .when(col("age") > 50, "50+")
)

# Group by 'age_group' and 'category' and count the number of posts in each category per age group
df_category_count = df_with_age_group.groupBy("age_group", "category") \
                                     .agg(count("category").alias("category_count"))

# Define a window specification to rank categories by count within each age group
window_spec = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Apply the window function to add a rank column
df_ranked = df_category_count.withColumn("rank", row_number().over(window_spec))

# Filter to keep only the most popular category per age group (rank == 1)
df_most_popular = df_ranked.filter(col("rank") == 1).drop("rank")

# Display the final result
display(df_most_popular)

In [ ]:
# M7: Task 8
# Find the median follower count for different age groups

# Group by 'age_group' and calculate the median follower count
df_median_follower = df_with_age_group.groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the final result
display(df_median_follower)

In [ ]:
# M7: Task 9
# Find how many users have joined between 2015-2020

# Extract the year from the 'date_joined' column to create 'post_year' column
df_with_year = df_user_cleaned.withColumn("post_year", year(col("date_joined")))

# Filter the data to include only entries between 2015 and 2020
df_filtered = df_with_year.filter(col("post_year").between(2015, 2020))

# Group by 'post_year' and count the number of users joined each year
df_users_joined = df_filtered.groupBy("post_year") \
                             .agg(count("ind").alias("number_users_joined"))

# Display the final result
display(df_users_joined)

In [ ]:
# M7: Task 10
# Find the media follower count of users that have joined between 2015-2020

# Join df_pin_cleaned and df_user_cleaned on the 'ind' column
df_combined = df_pin_cleaned.join(df_user_cleaned, on="ind", how="inner")

# Extract the year from the 'date_joined' column to create 'post_year' column
df_with_year = df_combined.withColumn("post_year", year(col("date_joined")))

# Filter the data to include only entries between 2015 and 2020
df_filtered = df_with_year.filter(col("post_year").between(2015, 2020))

# Group by 'post_year' and calculate the median follower count
df_median_follower = df_filtered.groupBy("post_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the final result
display(df_median_follower)

In [ ]:
# M7: Task 11
# Find the median follower count of users that have joined between 2015 and 2020, based on age group.

# Extract the year from the 'date_joined' column to create 'post_year' column
df_with_year = df_combined.withColumn("post_year", year(col("date_joined")))

# Create an 'age_group' column based on the 'age' column
df_with_age_group = df_with_year.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .when(col("age") > 50, "50+")
)

# Filter the data to include only entries between 2015 and 2020
df_filtered = df_with_age_group.filter(col("post_year").between(2015, 2020))

# Group by 'age_group' and 'post_year' and calculate the median follower count
df_median_follower = df_filtered.groupBy("age_group", "post_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the final result
display(df_median_follower)